In [2]:
import numpy as np
from scipy.spatial import KDTree


In [37]:
def nearest_neighbor_distance(X, Y):
    """
    Find the nearest (Euclidean) neighbor in B for each point in Ref
    A: Nx2 numpy array of source 2D points
    B: Nx2 numpy array of destination 2D point
    Returns:
    distances: Euclidean distances (errors) of the nearest neighbor
    indices: index of the nearest neighbor in B for each point in A
    """
    tree = KDTree(Y)
    distances, indices = tree.query(X) ##like we are looping in each point in X map it to Y
    return distances, indices




X = np.array( [ [0, 1], [0, 10]])
Y = np.array(  [[0, 14], [0, 5]])

distances, indices = nearest_neighbor_distance(X,Y)

Y= Y[indices] #the new mapping

print (  Y )     

[[ 0  5]
 [ 0 14]]
